# 11-05 Multi-Kernel 1D CNN으로 네이버 영화 리뷰 분류하기

## 네이버 영화 데이터 수집 & 전처리

모든 전처리는 RNN을 이용한 텍스트 분류 챕터의 네이버 영화 리뷰 분류하기( https://wikidocs.net/44249 )와 동일하게 수행

## Multi-Kernel 1D CNN으로 네이버 영화 리뷰 분류하기

케라스에서 다수의 커널을 사용할 경우에는 Funtional API를 사용하여 구현

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 128
dropout_ratio = (0.5, 0.8)
num_filters = 128
hidden_units = 128

model_input = Input(shape = (max_len,))
z = Embedding(vocab_size, embedding_dim, input_length = max_len, name="embedding")(model_input)
z = Dropout(dropout_ratio[0])(z)

conv_blocks = []

for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                    kernel_size = sz,
                    padding = "valid",
                    activation = "relu",
                    strides = 1)(z)
    conv = GlobalMaxPooling1D()(conv)
    conv_blocks.append(conv)

z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z = Dropout(dropout_ratio[1])(z)
z = Dense(hidden_units, activation="relu")(z)
model_output = Dense(1, activation="sigmoid")(z)

model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('CNN_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2, verbose=2, callbacks=[es, mc])

In [ ]:
loaded_model = load_model('CNN_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## 리뷰 예측해보기

RNN을 이용한 텍스트 분류 챕터의 네이버 영화 리뷰 분류하기 실습( https://wikidocs.net/44249 )에서 사용한 동일한 예측 함수 sentiment_predict를 사용

In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')